# Building a plotly choropleth map with slider
The goal is to combine plotly's choropleth map (https://plot.ly/python/choropleth-maps/) with the slider example (https://plot.ly/python/gapminder-example/). 

First I searched plotly's community forum and the web for an example of a plotly choropleth map with grid data or a slider without success. 

While I managed to replicate the slider animation example with my data (https://plot.ly/~mathias.riechert/59), adapting the choropleth code to access grid data with a slider does not seem to work.

In contrast to a regular map (like in example one), data is not loaded from a dataframe column but is uploaded to a plotly grid. In example two we can see that instead of regular x,y and text values in the data_dict, xsrc, ysrc and textsrc are used to access the grid data. 

    data_dict = {
           'xsrc': grid.get_column_reference(col_name_template.format(
                year=year, continent=continent, header='lifeExp'
            )),
            'ysrc': grid.get_column_reference(col_name_template.format(
                year=year, continent=continent, header='gdpPercap'
            )),


Chloropleth maps are defined in plotly by setting locations, z-values and text values in list form. In order to check if plotly choropleth maps support grid data, I searched in the api-documentation (https://plot.ly/python/reference/#choropleth) and found equivalents: locationsrc, zsrc, and textsrc. Therefore using grid data should be possible. When I draw my adapted graph though, I see only the coastline and the slider but no countries being highlighted. 

## Load data

In [25]:
import pandas as pd
import plotly   
import plotly.plotly as py
import time
from plotly.grid_objs import Grid, Column
plotly.__version__

'1.13.0'

In [23]:
dataset_original=pd.read_csv("data_paper_country.csv",sep=";")

dataset_original=dataset_original.drop(dataset_original.columns[[0]], axis=1)

dataset_original

,DOI,PUBYEAR,AUTHOR_CNT,REF_CNT,COUNTRY_CNT,INST_CNT,CIT_CNT,COUNTRYCODE,keyword
0,10.1002/ad.1700,2014,1.0,6.0,1.0,1.0,0.0,GBR,big data
1,10.1002/ad.1700,2014,1.0,6.0,1.0,1.0,0.0,GBR,big data
2,10.3305/nh.2014.29.4.7275,2014,4.0,72.0,1.0,2.0,1.0,ESP,big data
3,10.3305/nh.2014.29.4.7275,2014,4.0,72.0,1.0,2.0,1.0,ESP,big data
4,10.3305/nh.2014.29.4.7275,2014,4.0,72.0,1.0,2.0,1.0,ESP,big data
5,10.3305/nh.2014.29.4.7275,2014,4.0,72.0,1.0,2.0,1.0,ESP,big data
6,10.3305/nh.2014.29.4.7275,2014,4.0,72.0,1.0,2.0,1.0,ESP,big data
7,10.3305/nh.2014.29.4.7275,2014,4.0,72.0,1.0,2.0,1.0,ESP,big data
8,10.1016/j.amepre.2011.01.006,2011,3.0,63.0,1.0,1.0,4.0,USA,big data
9,10.1016/j.amepre.2011.01.006,2011,3.0,63.0,1.0,1.0,4.0,USA,big data


## Group data by pubyear and countrycode

In [9]:
#group by year and country:
grouped=dataset_original.groupby(['PUBYEAR','COUNTRYCODE'])

In [13]:
#aggregate to mean, count rows per group:
dataset=grouped.agg('mean')\
             .rename(columns = lambda x: x + ' mean')\
             .join(pd.DataFrame(grouped.size(), 
                                columns=['NUM_PAPERS'])).reset_index()
dataset

,PUBYEAR,COUNTRYCODE,Unnamed: 0 mean,AUTHOR_CNT mean,REF_CNT mean,COUNTRY_CNT mean,INST_CNT mean,CIT_CNT mean,NUM_PAPERS
0,2009,GBR,322.600000,4.000000,17.000000,1.000000,1.000000,12.000000,5
1,2010,CAN,114.500000,1.000000,41.000000,1.000000,1.000000,7.000000,2
2,2010,USA,38.500000,8.000000,54.000000,1.000000,7.000000,6.000000,10
3,2011,CHE,344.000000,2.000000,151.000000,2.000000,2.000000,16.000000,3
4,2011,USA,660.562500,2.687500,37.000000,1.062500,1.687500,3.375000,16
5,2012,AUS,1217.000000,5.000000,53.000000,2.000000,3.000000,23.000000,3
6,2012,CAN,2682.500000,17.000000,1.000000,2.000000,2.000000,1.000000,2
7,2012,CHN,590.238095,6.285714,26.142857,1.714286,2.857143,8.142857,21
8,2012,DEU,649.500000,1.000000,4.000000,1.000000,1.000000,0.000000,2
9,2012,FRA,2028.500000,5.000000,35.000000,2.000000,2.000000,7.000000,2


## build grid and upload to plotly

In [26]:

years_from_col = set(round(dataset['PUBYEAR'],0))
years_ints = sorted(list(years_from_col))
years = [str(year) for year in years_ints]
#years.remove('2009')

# make list of keywords
countries = []
for country in dataset['COUNTRYCODE']:
    if country not in countries: 
        countries.append(country)
        
columns = []
# make grid
for year in years:
    for country in countries:
        dataset_by_year = dataset[dataset['PUBYEAR'] == int(year)]
        dataset_by_year_and_country= dataset_by_year[dataset_by_year['COUNTRYCODE'] == country]
        for col_name in dataset_by_year_and_country:
            # each column name is unique
            column_name = '{year}_{country}_{header}_grid'.format(
                year=year, country=country, header=col_name
            )
            a_column = Column(list(dataset_by_year_and_country[col_name]), column_name)
            columns.append(a_column)

# upload grid
grid = Grid(columns)
url = py.grid_ops.upload(grid, 'grid'+str(time.time()), auto_open=False)
url

'https://plot.ly/~mathias.riechert/71/'

## Build visualization and upload

In [27]:
CONSTz='NUM_PAPERS'



#make figure:
figure = {
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True}
}

# fill in most of layout
figure['layout']['title'] = 'Paper with "big data" keywords per country'
figure['layout']['geo'] = {'showframe':False, 'showcoastlines':True, 'projection':{'type':'Mercator'}}
figure['layout']['hovermode'] = 'closest'
figure['layout']['margin'] = {'r': 250}
figure['layout']['slider'] = {
    'args': [
        'slider.value', {
            'duration': 700,
            'ease': 'cubic-in-out'
        }
    ],
    'initialValue': '2009',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': True},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]


sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

col_name_template = '{year}_{country}_{header}_grid'

year = 2009
for country in countries:
    data_dict = {
        'type':'choropleth',
        'locationsrc':grid.get_column_reference(col_name_template.format(
            year=year, country=country, header='COUNTRYCODE')),
        'zsrc':grid.get_column_reference(col_name_template.format(
            year=year, country=country, header=CONSTz)),
        'textsrc': grid.get_column_reference(col_name_template.format(
            year=year, country=country, header='COUNTRYCODE')),
        'colorscale':[[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        'autocolorscale':False,
        'reversescale':True,
        'marker': {
            'line':{
                'color': 'rgb(180,180,180)',
                'width':0.5           
            }},
            'colorbar':{
                'autotick':False,
                'title':'#paper'
            }
        }
    figure['data'].append(data_dict)





In [28]:
# check if there is meaningful content in data:
figure['data']

[{'autocolorscale': False,
  'colorbar': {'autotick': False, 'title': '#paper'},
  'colorscale': [[0, 'rgb(5, 10, 172)'],
   [0.35, 'rgb(40, 60, 190)'],
   [0.5, 'rgb(70, 100, 245)'],
   [0.6, 'rgb(90, 120, 245)'],
   [0.7, 'rgb(106, 137, 247)'],
   [1, 'rgb(220, 220, 220)']],
  'locationsrc': 'mathias.riechert:71:e6a26b',
  'marker': {'line': {'color': 'rgb(180,180,180)', 'width': 0.5}},
  'reversescale': True,
  'textsrc': 'mathias.riechert:71:e6a26b',
  'type': 'choropleth',
  'zsrc': 'mathias.riechert:71:4736e5'},
 {'autocolorscale': False,
  'colorbar': {'autotick': False, 'title': '#paper'},
  'colorscale': [[0, 'rgb(5, 10, 172)'],
   [0.35, 'rgb(40, 60, 190)'],
   [0.5, 'rgb(70, 100, 245)'],
   [0.6, 'rgb(90, 120, 245)'],
   [0.7, 'rgb(106, 137, 247)'],
   [1, 'rgb(220, 220, 220)']],
  'locationsrc': 'mathias.riechert:71:1e01ca',
  'marker': {'line': {'color': 'rgb(180,180,180)', 'width': 0.5}},
  'reversescale': True,
  'textsrc': 'mathias.riechert:71:1e01ca',
  'type': 'chorop

In [30]:
#locationsrc, zrsc and textsrc seem to have correct values --> let's look at a sample:

col_name_template.format(
            year=year, country=country, header=CONSTz)

'2009_URY_NUM_PAPERS_grid'

In [31]:
#looks good, now check if the resulting id fits:
grid.get_column_reference(col_name_template.format(
            year=year, country=country, header=CONSTz))


'mathias.riechert:71:5b5d26'

In [32]:

for year in years:
    frame = {'data': [], 'name': str(year)}
    for country in countries:
        data_dict = {
            'type':'choropleth',
            'locationsrc':grid.get_column_reference(col_name_template.format(
                year=year, country=country, header='COUNTRYCODE')),
            'zsrc':grid.get_column_reference(col_name_template.format(
                year=year, country=country, header=CONSTz)),
            'textsrc': grid.get_column_reference(col_name_template.format(
                year=year, country=country, header='COUNTRYCODE')),
            'colorscale':[[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
            'autocolorscale':False,
            'reversescale':True,
            'marker': {
                'line':{
                    'color': 'rgb(180,180,180)',
                    'width':0.5           
                }},
                'colorbar':{
                    'autotick':False,
                    'title':'#paper'
                }
            }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': True},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]
py.icreate_animations(figure, 'BigDataPapersWorldMap'+str(time.time()))

no fills, only the years. Maybe the colorscale has to be set differently?

The map can be found here: https://plot.ly/~mathias.riechert/72